In [1]:
import os
import sklearn
import pandas as pd
import numpy as np
from __future__ import division
import collections
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import urllib
from sklearn.svm import LinearSVC
from stats import Stats

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Folder and path parameters

In [2]:
data_dir = "./Data"
kaggle_dir = os.path.join(data_dir, "Kaggle")
kenpom_dir = os.path.join(data_dir, "Kenpom")
sr_dir = os.path.join(data_dir, "Sports_ref")

In [3]:
teams_pd = pd.read_csv(os.path.join(kaggle_dir, "Teams.csv"))
teams_list = teams_pd["TeamName"].tolist()
print(teams_list[0:10])

['Abilene Chr', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama St', 'Albany NY', 'Alcorn St', 'Alliant Intl', 'American Univ']


## Using Pandas to visualize data

In [4]:
reg_season_compact_pd = pd.read_csv(kaggle_dir + "/RegularSeasonCompactResults.csv")
reg_season_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [5]:
reg_season_detailed_pd = pd.read_csv(kaggle_dir + '/RegularSeasonDetailedResults.csv')
reg_season_detailed_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [6]:
seasons_pd = pd.read_csv(kaggle_dir + '/Seasons.csv')
seasons_pd.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [7]:
teams_pd = pd.read_csv(kaggle_dir + '/Teams.csv')
teamList = teams_pd['TeamName'].tolist()
print(teams_pd.head())
teamList[1]

   TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2019
1    1102    Air Force           1985          2019
2    1103        Akron           1985          2019
3    1104      Alabama           1985          2019
4    1105  Alabama A&M           2000          2019


'Air Force'

In [8]:
sr_pd = pd.read_csv(sr_dir + "/2002_basic.csv", skiprows=1)
sr_pd.head()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Air Force,28,9,19,0.321,-0.95,3.23,3.0,11.0,...,331,465,0.712,154.0,614,327,228,39,390,NaN
1,2,Akron,31,10,21,0.323,-4.84,0.36,5.0,13.0,...,554,731,0.758,295.0,933,416,180,77,450,NaN
2,3,Alabama A&M,29,19,10,0.655,-11.33,-15.81,12.0,6.0,...,478,711,0.672,506.0,1217,426,395,116,504,NaN
3,4,Alabama-Birmingham,30,13,17,0.433,5.56,2.96,6.0,10.0,...,428,623,0.687,376.0,1117,433,231,87,448,NaN
4,5,Alabama State,32,19,13,0.594,-13.25,-14.15,12.0,6.0,...,483,755,0.640,389.0,1167,314,217,89,538,NaN


In [9]:
sr_pd = pd.read_csv(sr_dir + "/2002_adv.csv", skiprows=1)
sr_pd.head()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,1,Air Force,28,9,19,0.321,-0.95,3.23,3.0,11.0,...,0.485,0.585,42.3,61.4,NaN,3.2,0.553,22.1,NaN,0.288
1,2,Akron,31,10,21,0.323,-4.84,0.36,5.0,13.0,...,0.311,0.559,47.5,58.7,NaN,4.4,0.506,18.9,NaN,0.350
2,3,Alabama A&M,29,19,10,0.655,-11.33,-15.81,12.0,6.0,...,0.339,0.494,49.3,52.3,NaN,6.7,0.458,17.7,NaN,0.238
3,4,Alabama-Birmingham,30,13,17,0.433,5.56,2.96,6.0,10.0,...,0.332,0.512,50.5,58.6,NaN,5.2,0.477,17.9,NaN,0.243
4,5,Alabama State,32,19,13,0.594,-13.25,-14.15,12.0,6.0,...,0.317,0.514,51.2,43.9,NaN,4.9,0.479,20.9,NaN,0.287


In [10]:
kenpom_pd = pd.read_csv(kenpom_dir + "/summaries/summary02.csv")
kenpom_pd.head()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM
0,2002,Air Force,55.7833,327,54.1733,327,99.4506,190,103.8170,149,106.8100,283,105.4580,214,-1.64046,181
1,2002,Akron,66.7009,260,65.6394,251,102.6500,128,105.2430,130,110.8370,313,111.1810,295,-5.93802,227
2,2002,Alabama,69.7639,171,68.0142,170,108.2290,42,114.0460,24,96.0457,56,95.3126,32,18.73300,22
3,2002,Alabama A&M,81.6407,1,78.9833,1,97.8208,220,92.1604,304,90.6649,4,101.6930,136,-9.53265,262
4,2002,Alabama St.,68.1969,223,63.0507,305,94.9003,264,90.1231,312,93.9840,26,102.7400,159,-12.61680,272


In [11]:
kenpom_pd = pd.read_csv(kenpom_dir + "/Misc_stats/misc02.csv")
kenpom_pd.head()

,Season,TeamName,FG2Pct,RankFG2Pct,FG3Pct,RankFG3Pct,FTPct,RankFTPct,BlockPct,RankBlockPct,...,RankOppF3GRate,ARate,RankARate,OppARate,RankOppARate,StlRate,RankStlRate,OppStlRate,RankOppStlRate,DefensiveFingerprint
0,2002,Air Force,54.8061,4,37.2760,58,71.1828,92,6.8287,235,...,64,61.3508,48,47.3958,11,0.1412,11,0.1084,218,18.6
1,2002,Akron,48.4271,136,37.0600,62,75.7576,12,6.9726,226,...,201,57.3508,115,55.3594,167,0.0868,263,0.1140,260,2.0
2,2002,Alabama,50.6579,56,30.8677,290,73.5782,40,8.0468,176,...,191,50.5495,267,46.9880,8,0.1053,130,0.0881,53,7.9
3,2002,Alabama A&M,44.2636,284,31.5871,272,66.4157,253,9.4669,107,...,176,52.7370,217,52.4425,92,0.1665,1,0.1039,175,10.2
4,2002,Alabama St.,45.8801,244,33.4646,209,64.4444,286,5.7722,280,...,9,44.2424,315,48.7465,32,0.0985,181,0.1358,321,13.4


# Generating Necessary Dataframes

In [12]:
reg_season_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'RegularSeasonCompactResults.csv'))
reg_season_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [13]:
conference_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'ConferenceTourneyGames.csv'))
conference_tourney_compact_pd.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,121,1194,1144
1,2001,a_sun,121,1416,1240
2,2001,a_sun,122,1209,1194
3,2001,a_sun,122,1359,1239
4,2001,a_sun,122,1391,1273


In [14]:
NCAA_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'NCAATourneyCompactResults.csv'))
NCAA_tourney_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [15]:
secondary_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'NCAATourneyCompactResults.csv'))
secondary_tourney_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [16]:
teams_pd = pd.read_csv(os.path.join(kaggle_dir,'teams.csv'))
teamList = teams_pd['TeamName'].tolist()
teams_pd.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2019
1,1102,Air Force,1985,2019
2,1103,Akron,1985,2019
3,1104,Alabama,1985,2019
4,1105,Alabama A&M,2000,2019


In [17]:
conferences_pd = pd.read_csv(os.path.join(kaggle_dir,'conferences.csv'))
conferences_pd.head()

,ConfAbbrev,Description
0,a_sun,Atlantic Sun Conference
1,a_ten,Atlantic 10 Conference
2,aac,American Athletic Conference
3,acc,Atlantic Coast Conference
4,aec,America East Conference


In [27]:
team_conferences_pd = pd.read_csv(os.path.join(kaggle_dir,'TeamConferences.csv'))
team_conferences_pd.head()

,Season,TeamID,ConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


# Functions below are for getting/checking specific stats

In [29]:
# init stats instance using dictionary of dataframes
dfs = {"teams_pd": teams_pd, 
       "team_conferences_pd": team_conferences_pd, 
       "kenpom_pd": kenpom_pd, 
       "sr_pd": sr_pd}
stats = Stats(dfs)

### Create a dictionary of all the stat getters

In [ ]:
stat_getters = {}
# stat_getters[0] = getWinPercentage
# stat_getters[1] = checkPower6
# stat_getters[2] = getTrueShooting
# stat_getters[3] = getTempo
# stat_getters[4] = get3pm
# stat_getters[5] = getPointPercentageFrom3pt
# stat_getters[6] = getAdjDefensiveEfficiency
# stat_getters[7] = getAdjOffensiveEfficiency
# stat_getters[8] = getFTPercentage
# stat_getters[9] = getFTRate
# stat_getters[10] = getORPercentage
# stat_getters[11] = getOppORPercentage
# stat_getters[11] = getOppEfg
# stat_getters[12] = getSize
# stat_getters[13] = getSRS
# stat_getters[14] = getExperience
# stat_getters[15] = getAssistRate
# stat_getters[16] = getPpg
stat_getters[0] = stats.checkPower6
stat_getters[1] = stats.get2PtFGPercentage
stat_getters[2] = stats.get3PtFGPercentage
stat_getters[3] = stats.get3pPercent
stat_getters[4] = stats.get3pm
stat_getters[5] = stats.getAdjDefensiveEfficiency
stat_getters[6] = stats.getAdjEfficiencyMargin
stat_getters[7] = stats.getAdjOffensiveEfficiency
stat_getters[8] = stats.getAdjTempo
stat_getters[9] = stats.getAssistRate
stat_getters[10] = stats.getBenchMin
stat_getters[11] = stats.getBlockPercentage
stat_getters[12] = stats.getDefensiveEfficiency
stat_getters[13] = stats.getDefensiveFingerprint
stat_getters[14] = stats.getEffectiveHeight
stat_getters[15] = stats.getEfg
stat_getters[16] = stats.getExperience
stat_getters[17] = stats.getFTPercentage
stat_getters[18] = stats.getFTRate
stat_getters[19] = stats.getFgPercent
stat_getters[20] = stats.getFtPercent
stat_getters[21] = stats.getFtm
stat_getters[22] = stats.getNumWins
stat_getters[23] = stats.getORPercentage
stat_getters[24] = stats.getOffensiveEfficiency
stat_getters[25] = stats.getOpp2PtFGPercentage
stat_getters[26] = stats.getOpp3PtFGPercentage
stat_getters[27] = stats.getOppAssistRate
stat_getters[28] = stats.getOppBlockPercentage
stat_getters[29] = stats.getOppEfg
stat_getters[30] = stats.getOppFTPercentage
stat_getters[31] = stats.getOppFTRate
stat_getters[32] = stats.getOppORPercentage
stat_getters[33] = stats.getOppPointPercentageFrom2pt
stat_getters[34] = stats.getOppPointPercentageFrom3pt
stat_getters[35] = stats.getOppPointPercentageFromFT
stat_getters[36] = stats.getOppPpg
stat_getters[37] = stats.getOppStealRate
stat_getters[38] = stats.getOppTOPercentage
stat_getters[39] = stats.getPointPercentageFrom2pt
stat_getters[40] = stats.getPointPercentageFrom3pt
stat_getters[41] = stats.getPointPercentageFromFT
stat_getters[42] = stats.getPpg
stat_getters[43] = stats.getSRS
stat_getters[44] = stats.getSize
stat_getters[45] = stats.getStealRate
stat_getters[46] = stats.getTOPercentage
stat_getters[47] = stats.getTempo
stat_getters[46] = stats.getTotalReboundPercent
stat_getters[47] = stats.getTrueShooting
stat_getters[48] = stats.getWinPercentage

## Additional Game data getters

In [ ]:
def getLoc(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

# Getting the Season Vectors for each team and year :0

In [ ]:
#creates the stat vector for a given team during a give season/year
def getSeasonVector(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: '2002', but was: " + season
    
    team_vector = []
    for i in range(len(stat_getters)):
        team_vector.append(stat_getters.get(i)(team_name, season))
    return np.array(team_vector)
getSeasonVector("Auburn", "2019")

# Make a dictionary of every team and the respective season vector    

In [ ]:
def createSeasonDict(season):
    print("SEASON = {0}".format(season))
    seasonDict = {}
    for team in teams_list:
        team_vector = getSeasonVector(team, str(season))
        
        #if any of the entries in the vector equals 0, then the team name wasn't present in one of the data files
        if not any(map(lambda x: x == "missing", team_vector)):
            seasonDict[team] = team_vector
    return seasonDict

In [ ]:
#takes in a regularseason or a tourney df, returns the counter to keep track of training matrix index
def addGames(df, counter, team_vectors, xTrainSeason, yTrainSeason, neutral=False):
    for index, row in df.iterrows():
            #determine the difference vector for each game
            w_team = getTeamName(row['WTeamID'])
            l_team = getTeamName(row['LTeamID'])
            
            #make sure teams are in the seasonDictionary
            if team_vectors.get(w_team, np.array([False])).any() and team_vectors.get(l_team, np.array([False])).any():
                w_vector = team_vectors[w_team]
                l_vector = team_vectors[l_team]
                diff_vector = [a - b for a, b in zip(w_vector, l_vector)]
                
                if neutral:  
                    diff_vector.append(0)
                else:
                    diff_vector.append(getLoc(row['WLoc']))
                    
                #flip every odd numbered game
                if (counter % 2 == 0):
                    #predict the winning team
                    xTrainSeason[counter] = diff_vector
                    yTrainSeason[counter] = 1
                else:
                    #predict the losing team
                    xTrainSeason[counter] = [-x for x in diff_vector]
                    yTrainSeason[counter] = 0
                counter += 1
    return counter

In [ ]:
def createTrainingSet(years):
    #calculating the dimensions of the training sets
    totalNumGames = 0
    for year in years:
        reg_season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        NCAA_tourney = NCAA_tourney_compact_pd[NCAA_tourney_compact_pd['Season'] == year]
        secondary_tourney = secondary_tourney_compact_pd[secondary_tourney_compact_pd['Season'] == year]
        conference_tourney = conference_tourney_compact_pd[conference_tourney_compact_pd['Season'] == year]
        totalNumGames += (len(reg_season.index) + len(conference_tourney.index) + len(NCAA_tourney.index) + len(secondary_tourney.index))
        
    statsLength = len(stat_getters) + 1
    xTrain = np.zeros((totalNumGames, statsLength))
    yTrain = np.zeros((totalNumGames))
    seasonCounter = 0
    
    #create a training set for each season to put into the x and y Train
    for year in years:
        #get the team vectors for the year in iteration
        team_vectors = createSeasonDict(year)
        
        #get all of the games for the year in iteration
        reg_season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        NCAA_tourney = NCAA_tourney_compact_pd[NCAA_tourney_compact_pd['Season'] == year]
        secondary_tourney = secondary_tourney_compact_pd[secondary_tourney_compact_pd['Season'] == year]
        conference_tourney = conference_tourney_compact_pd[conference_tourney_compact_pd['Season'] == year]
        
        games_in_season = len(reg_season.index) + len(NCAA_tourney.index) + len(secondary_tourney.index) + len(conference_tourney.index)
        xTrainSeason = np.zeros((games_in_season, statsLength))
        yTrainSeason = np.zeros((games_in_season))
        counter = 0

        #add each game to the training vectors from the regular season
        counter = addGames(reg_season, counter, team_vectors, xTrainSeason, yTrainSeason, neutral=False)
            
        #add each game to the training vector from the NCAA tourney; games are always neutral    
        counter = addGames(NCAA_tourney, counter, team_vectors, xTrainSeason, yTrainSeason, neutral=True)
            
        #add each game to the training vector from the secondary tourneys
        counter = addGames(secondary_tourney, counter, team_vectors, xTrainSeason, yTrainSeason, neutral=False)

        xTrain[seasonCounter:games_in_season+seasonCounter] = xTrainSeason
        yTrain[seasonCounter:games_in_season+seasonCounter] = yTrainSeason
        seasonCounter += games_in_season
    return xTrain, yTrain

In [ ]:
# years = range(2002, 2020)
# xTrain, yTrain = createTrainingSet(years)
# np.save('xTrain2019_David1.npy', xTrain)
# np.save('yTrain2019_David1.npy', yTrain)

In [ ]:
xTrain = np.load("xTrain2019_David2.npy")
yTrain = np.load("yTrain2019_David2.npy")

In [ ]:
# takes out the 47th index twice to remove NaNs. Never really figured out why they were there...
# run this cell twice

print(xTrain.shape)
print(xTrain[0])
temp = []
for i in range(xTrain.shape[0]):
#     xTrain[i] = np.concatenate((xTrain[i][:47], xTrain[i][48:]), axis=0)
    temp.append(np.delete(xTrain[i], [47, 48]))
xTrain = np.array(temp)
print(xTrain.shape)
print(xTrain[0])

In [ ]:
print(xTrain.shape)
count = 0
for i in range(xTrain.shape[0]):
    if not np.all(np.isfinite(xTrain[i])):
        count += 1
        print(i)
        print(xTrain[i])
print(count)

In [ ]:
print(yTrain.shape)
print(yTrain[0:30])
print(xTrain.shape)

In [ ]:
model = GradientBoostingRegressor(n_estimators=100, max_depth=5)

In [ ]:
def showDependency(predictions, test, stat, my_categories):
    difference = test[:,my_categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Probability of Team 1 Win')
    plt.xlabel(stat + ' Difference (Team 1 - Team 2)')
    plt.show()

In [ ]:
def showFeatureImportance(my_categories):
    fx_imp = pd.Series(model.feature_importances_, index=my_categories)
    fx_imp /= fx_imp.max()
    fx_imp.sort_values()
    fx_imp.plot(kind='barh')

# Accuracy of V1: 83.12% 
## Accuracy of V2: 82.64%

In [ ]:
categories=['NumWins','Effective FG%','Opponent Effective FG%','ORebound %','Opponent ORebound%', 'TO%','Opponent TO%','FT Rate',
            'Opponent FT rate', 'Adj Defensive Efficiency','Adj Offensive Efficiency', 'Adj Tempo', 'Effective Height', 
            'Experience', 'Bench Min', 'Home']
accuracy=[]

for i in range(1):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
    print("Finished iteration:", i)
print("The accuracy is", sum(accuracy)/len(accuracy))

In [ ]:
def predictGame(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict([diff]) 
    #return model.predict_proba([diff])

In [ ]:
team1_name = "Belmont"
team2_name = "Temple"
team1_vector = getSeasonVector(team1_name, "2019")
team2_vector = getSeasonVector(team2_name, "2019")
print(team1_vector)
print(team2_vector)
print('Probability that ' + team1_name + ' wins:', predictGame(team1_vector, team2_vector, 0)[0])

In [ ]:
showFeatureImportance(categories)

# Everything below is for making a bracket

In [ ]:
import random
def winnerIn5Games(team1, team2):
    team1_vector = getSeasonVector(team1, "2019")
    team2_vector = getSeasonVector(team2, "2019")
    team1_win_chance = predictGame(team1_vector, team2_vector, 0)[0]
    team1_winCount = 0;
    for i in range(5):
        randomProb = random.random()
        if team1_win_chance > randomProb:
            team1_winCount += 1
            
    print(team1 + " wins", team1_winCount, " games out of 5")

In [ ]:
winnerIn5Games("Duke", "N Dakota St")
winnerIn5Games("VA Commonwealth", "UCF")
winnerIn5Games("Mississippi St", "Liberty")
winnerIn5Games("Virginia Tech", "St Louis")
winnerIn5Games("Maryland", "Belmont")
winnerIn5Games("LSU", "Yale")
winnerIn5Games("Louisville", "Minnesota")
winnerIn5Games("Michigan St", "Bradley")
winnerIn5Games("Gonzaga", "F Dickinson")
winnerIn5Games("Syracuse", "Baylor")
winnerIn5Games("Marquette", "Murray St")
winnerIn5Games("Florida St", "Vermont")
winnerIn5Games("Buffalo", "Arizona St")
winnerIn5Games("Texas Tech", "N Kentucky")
winnerIn5Games("Nevada", "Florida")
winnerIn5Games("Michigan", "Montana")
winnerIn5Games("Virginia", "Gardner Webb")
winnerIn5Games("Mississippi", "Oklahoma")
winnerIn5Games("Wisconsin", "Oregon")
winnerIn5Games("Kansas St", "UC Irvine")
winnerIn5Games("Villanova", "St Mary's CA")
winnerIn5Games("Purdue", "Old Dominion")
winnerIn5Games("Cincinnati", "Iowa")
winnerIn5Games("Tennessee", "Colgate")
winnerIn5Games("North Carolina", "Iona")
winnerIn5Games("Utah St", "Washington")
winnerIn5Games("Auburn", "New Mexico St")
winnerIn5Games("Kansas", "Northeastern")
winnerIn5Games("Iowa St", "Ohio St")
winnerIn5Games("Houston", "Georgia St")
winnerIn5Games("Wofford", "Seton Hall")
winnerIn5Games("Kentucky", "Abilene Chr")

In [ ]:
winnerIn5Games("Duke", "Virginia Tech")
winnerIn5Games("LSU", "Michigan St")
winnerIn5Games("Virginia", "Wisconsin")
winnerIn5Games("Purdue", "Tennessee")
winnerIn5Games("Gonzaga", "Florida St")
winnerIn5Games("Texas Tech", "Michigan")
winnerIn5Games("North Carolina", "Auburn")
winnerIn5Games("Houston", "Seton Hall")

In [ ]:
winnerIn5Games("Duke", "Michigan St")
winnerIn5Games("Wisconsin", "Tennessee")
winnerIn5Games("Gonzaga", "Michigan")
winnerIn5Games("North Carolina", "Houston")

In [ ]:
winnerIn5Games("Duke", "Gonzaga")
winnerIn5Games("Tennessee", "North Carolina")

In [ ]:
winnerIn5Games("Gonzaga", "North Carolina")